In [8]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction
using Missings

os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")


PyCall.pygui(:tk)

FONTSIZE=20.5;

In [2]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
###############################                Parameter Learning Tables             ###########################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [25]:
function get_up_low(post_dict, upper_quant, lower_quant, γ0_is_log)
    p = [upper_quant,lower_quant]
    quantiles = zeros(19, 6, 2)
    sorted_keys = sort(collect(keys(post_dict)))
    # Adjust for when gamma_0 is considered on the log scale

        for i in 1:19
            mat = post_dict[ sorted_keys[i] ]
            quantiles[i, 1,:] = quantile( mat[:,1], p )
            quantiles[i, 2,:] = quantile( mat[:,2], p )
            quantiles[i, 3,:] = quantile( mat[:,3], p )
            quantiles[i, 4,:] = quantile( mat[:,4], p )
            quantiles[i, 5,:] = (γ0_is_log) ? quantile( np.log(mat[:,5]), p ) : quantile( mat[:,5], p )
            quantiles[i, 6,:] = quantile( mat[:,6], p )
        end

    return quantiles
end

Realizations = [string(i) for i in 1:100];
cal_years = collect(range(2015,step=15,length=20));

# Load the present day posteriors 
gamma_0_is_log = true
present_posterior = np.load("../Data/Training_Data/posterior_samples_All_Combined.npy");

# Compute upper and lower quantile bounds for present day posteriors
present_bounds = []
present_widths = []
for i in 1:6
    lower = (i == 5 && gamma_0_is_log) ? quantile(np.log(present_posterior[:,i]), 0.05) : quantile(present_posterior[:,i], 0.05)
    upper = (i == 5 && gamma_0_is_log) ? quantile(np.log(present_posterior[:,i]), 0.95) : quantile(present_posterior[:,i], 0.95)
    push!(present_bounds, (lower,upper))
    push!(present_widths, upper - lower)
end
present_bounds = reshape(present_bounds, 1, :)
present_widths = reshape(present_widths, 1, :);


# Change to wherever your posterior dictionaries are located
path_to_posteriors = "../Data/Posterior_Data"

# Initialize empty vectors to hold interval bounds and widths 
all_widths = zeros(length(Realizations),19,6)
all_bounds = zeros(length(Realizations), 19, 6, 2)


# Loop to calculate quantiles and credible interval widths
    for (iter, r) in enumerate(Realizations)
        post = JLD2.load("$(path_to_posteriors)/R_$(r)_Posterior_Dict.jld2", "post_data")
        cred_int_90_up_low = get_up_low(post, 0.95, 0.05,gamma_0_is_log)
        all_bounds[iter, :, :, :] = cred_int_90_up_low
        all_widths[iter,:,:] = cred_int_90_up_low[:,:,1] .- cred_int_90_up_low[:,:,2]
    end
# The average 5th percentile and and 95th percentiles of the six MALI parameters across all trajectories over the 19 years of calibration
avg_bounds = mean(all_bounds,dims=1)
avg_bounds = dropdims(avg_bounds, dims=1)

# The average 90% credible interval widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
avg_widths = mean(all_widths,dims=1)
avg_widths = dropdims(avg_widths, dims=1)
println(size(avg_widths))
# The 17% and 83% quantiles of the 90% cred int widths of the six MALI parameters across all trajectories for each of the 19 years of calibration
width_variation_interval = np.quantile(all_widths, 0.83,axis=0) .- np.quantile(all_widths, 0.17,axis=0)


# Store the bounds as tuples in one matrix
bounds_as_tuples = fill((0.0,0.0),(19,6))
for i in 1:19
    for j in 1:6
        bounds_as_tuples[i,j] = (avg_bounds[i,j,2], avg_bounds[i,j,1])
    end
end
# Add the 2015 90% credible interval bounds
bounds_w_present = vcat(present_bounds, bounds_as_tuples);

# Add the 2015 90% credible widths
widths_w_present = vcat(present_widths, avg_widths);


# Add the years of calibration as another column
bounds_w_years = hcat(cal_years, bounds_w_present)
widths_w_years = hcat(cal_years, widths_w_present)
width_var_w_years = hcat(cal_years[2:end], width_variation_interval)

# Dataframe column names
names = ["Latest_calibration_year","vmThresh","fricExp","mu_scale","stiff_scale","gamma0","melt_flux"]

# Covnert to datframe before saving to disc
params_df_bounds = DataFrame(bounds_w_years, names)
params_df_widths = DataFrame(widths_w_years, names)
params_df_width_var = DataFrame(width_var_w_years, names)

#Calculate total percent decrease of 90% cred interval width
pct_change = ((collect(params_df_widths[1,2:end]) .- collect(params_df_widths[20,2:end])) ./ collect(params_df_widths[1,2:end])) .* 100
# Make a 1×7 row: first element is label, then pct_change values
new_row = hcat("total_%_decrease", pct_change...)
# Convert to DataFrame with the same column names
new_df = DataFrame(new_row, names)
# Append to params_df_widths
append!(params_df_widths, new_df)


# CSV.write("../Data/Parameter_Learning_Tables/param_90_CI_bounds_log_gamma0_$(gamma_0_is_log).csv",params_df_bounds)
# CSV.write("../Data/Parameter_Learning_Tables/param_90_CI_widths_log_gamma0_$(gamma_0_is_log).csv",params_df_widths)
# CSV.write("../Data/Parameter_Learning_Tables/param_90_CI_variation_log_gamma0_$(gamma_0_is_log).csv",   params_df_width_var)

(19, 6)


Row,Latest_calibration_year,vmThresh,fricExp,mu_scale,stiff_scale,gamma0,melt_flux
,Any,Any,Any,Any,Any,Any,Any
1,2015,34089.3,0.172232,0.232868,0.205653,2.53179,30.4802
2,2030,33780.8,0.167235,0.21296,0.195873,2.46702,30.3705
3,2045,33765.3,0.15895,0.208421,0.190561,2.25623,30.1791
4,2060,33642.9,0.157167,0.206643,0.186847,1.8003,29.8141
5,2075,33475.8,0.153437,0.203493,0.181547,1.24637,28.1198
6,2090,33153.9,0.151732,0.198547,0.175937,1.2052,26.9038
7,2105,32615.3,0.145927,0.181701,0.164678,1.13852,26.5008
8,2120,32367.7,0.139435,0.175612,0.161105,1.02016,26.2051
9,2135,31971.1,0.135851,0.172647,0.154908,0.898344,24.2433


In [26]:
##CALCULATING THE YEAR BY YEAR CHANGE IN UNCERTAINTY RANGES FOR EACH PARAMETER, AS WELL AS PERCENT CONTRIBUTION TO TOTAL LEARNING
##
cal_years = collect(range(2015,step=15,length=20))

Δ_param_width = zeros(20, 6)
Δ_param_width_pct = zeros(20, 6)

#Total learning from 2015 - 2300

# find row indices
total_idx1 = only(findall(x -> x == 2015, cal_years))
total_idx2 = only(findall(x -> x == 2300, cal_years))

# extract vectors (dropping year column)
present = collect(params_df_widths[total_idx1, :])[2:end]
final = collect(params_df_widths[total_idx2, :])[2:end]

# compute differences
total_diff = final .- present
Δ_param_width[20, :] = total_diff
Δ_param_width_pct[20,:] .= 100.0

learn_periods = []

for i in 1:(length(cal_years)-1)
    yr1 = cal_years[i]
    yr2 = cal_years[i+1]

    # find row indices
    row_idx1 = only(findall(x -> x == yr1, cal_years))
    row_idx2 = only(findall(x -> x == yr2, cal_years))

    # extract vectors (dropping year column)
    row1 = collect(params_df_widths[row_idx1, :])[2:end]
    row2 = collect(params_df_widths[row_idx2, :])[2:end]

    # compute differences
    diff = row2 .- row1
    Δ_param_width[i, :] = diff
    Δ_param_width_pct[i, :] = (diff ./ total_diff) .* 100
    # Collect this learning period as string for datframe labels
    push!(learn_periods, "$(yr1) - $(yr2)")
end

# Add the total learning period
push!(learn_periods, "2015 - 2300")
width_change_mat = zeros(20,12)

# Put the parameter width changes and percent changes together param by param columnwise
for i in 1:6
    width_change_mat[:,(2*i)-1] = Δ_param_width[:,i]
    width_change_mat[:,(2*i)] = Δ_param_width_pct[:,i]
end


names = ["LearningPeriod","ΔvmThresh","Δ_%_vmThresh","Δ_fricExp","Δ_%_fricExp","Δ_mu_scale","Δ_%_mu_scale","Δ_stiff_scale","Δ_%_stiff_scale","Δ_gamma0","Δ_%_gamma0","Δ_melt_flux","Δ_%_melt_flux"]
width_change_w_years = hcat(learn_periods, width_change_mat)
df_bounds_yr_to_yr = DataFrame(width_change_w_years, names)
# CSV.write("../Data/Parameter_Learning_Tables/yr_to_yr_change_in_widths_log_gamma0_$(gamma_0_is_log).csv", df_bounds_yr_to_yr)

Row,LearningPeriod,ΔvmThresh,Δ_%_vmThresh,Δ_fricExp,Δ_%_fricExp,Δ_mu_scale,Δ_%_mu_scale,Δ_stiff_scale,Δ_%_stiff_scale,Δ_gamma0,Δ_%_gamma0,Δ_melt_flux,Δ_%_melt_flux
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,2015 - 2030,-308.545,3.69178,-0.00499717,3.30688,-0.0199084,13.9869,-0.00978005,8.45708,-0.0647679,2.83889,-0.109738,0.935563
2,2030 - 2045,-15.5338,0.185864,-0.00828424,5.4821,-0.00453845,3.18854,-0.00531249,4.59386,-0.210791,9.23934,-0.191358,1.63141
3,2045 - 2060,-122.379,1.46428,-0.00178388,1.18048,-0.00177861,1.24958,-0.00371431,3.21187,-0.455934,19.9844,-0.364953,3.11139
4,2060 - 2075,-167.098,1.99935,-0.00372934,2.46789,-0.00314946,2.21269,-0.00529938,4.58252,-0.55393,24.2797,-1.69436,14.4451
5,2075 - 2090,-321.917,3.85177,-0.00170556,1.12866,-0.00494637,3.47513,-0.00561054,4.85159,-0.0411684,1.80448,-1.21598,10.3668
6,2090 - 2105,-538.596,6.44437,-0.0058047,3.84127,-0.0168463,11.8356,-0.0112585,9.73551,-0.0666812,2.92275,-0.40298,3.43558
7,2105 - 2120,-247.567,2.96217,-0.00649159,4.29581,-0.00608864,4.27765,-0.00357312,3.08977,-0.118359,5.18788,-0.295708,2.52104
8,2120 - 2135,-396.591,4.74526,-0.00358477,2.37222,-0.00296495,2.08306,-0.00619757,5.35921,-0.121818,5.33949,-1.96177,16.725
9,2135 - 2150,-293.688,3.51401,-0.0040266,2.6646,-0.00290024,2.0376,-0.00486038,4.2029,-0.119725,5.24774,-0.744808,6.34982


In [46]:
println(sum(df_bounds_yr_to_yr[1:5, 11]) + sum(df_bounds_yr_to_yr[6:13, 11]) + sum(df_bounds_yr_to_yr[14:19, 11]))

100.00000000000001


In [43]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2015 to 2090: ")
    println(sum(df_bounds_yr_to_yr[1:5, col]))
end

Δ_%_vmThresh decrease from 2015 to 2090: 
11.193044442419605
Δ_%_fricExp decrease from 2015 to 2090: 
13.56602081348913
Δ_%_mu_scale decrease from 2015 to 2090: 
24.11285593408966
Δ_%_stiff_scale decrease from 2015 to 2090: 
25.69691638711342
Δ_%_gamma0 decrease from 2015 to 2090: 
58.146813524449946
Δ_%_melt_flux decrease from 2015 to 2090: 
30.490298144376688


In [47]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2090 to 2210: ")
    println(sum(df_bounds_yr_to_yr[6:13, col]))
end



Δ_%_vmThresh decrease from 2090 to 2210: 
65.35858638372444
Δ_%_fricExp decrease from 2090 to 2210: 
56.43968759374293
Δ_%_mu_scale decrease from 2090 to 2210: 
65.15053906260987
Δ_%_stiff_scale decrease from 2090 to 2210: 
61.183643628975034
Δ_%_gamma0 decrease from 2090 to 2210: 
36.681516511449466
Δ_%_melt_flux decrease from 2090 to 2210: 
56.939614177713494


In [48]:
cols = [3,5,7,9,11,13]
for (i,col) in enumerate(cols)
    println("$(names[2i+1]) decrease from 2210 to 2300: ")
    println(sum(df_bounds_yr_to_yr[14:19, col]))
end


Δ_%_vmThresh decrease from 2210 to 2300: 
23.448369173855948
Δ_%_fricExp decrease from 2210 to 2300: 
29.99429159276793
Δ_%_mu_scale decrease from 2210 to 2300: 
10.736605003300483
Δ_%_stiff_scale decrease from 2210 to 2300: 
13.119439983911551
Δ_%_gamma0 decrease from 2210 to 2300: 
5.171669964100604
Δ_%_melt_flux decrease from 2210 to 2300: 
12.570087677909815


In [27]:
##CALCULATING THE YEAR BY YEAR CHANGE IN UNCERTAINTY RANGE STANDARD DEVIATION FOR EACH PARAMETER, AS WELL AS PERCENT CONTRIBUTION TO TOTAL CHANGE
##
cal_years = collect(range(2030,step=15,length=19))

Δ_width_var = zeros(19, 6)
Δ_width_var_pct = zeros(19, 6)

#Total learning from 2015 - 2300

# find row indices
total_idx1 = only(findall(x -> x == 2030, cal_years))
total_idx2 = only(findall(x -> x == 2300, cal_years))

# extract vectors (dropping year column)
present = collect(params_df_width_var[total_idx1, :])[2:end]
final = collect(params_df_width_var[total_idx2, :])[2:end]

# compute differences
total_diff = final .- present
Δ_width_var[19, :] = round.(total_diff, sigdigits = 6) #20
Δ_width_var_pct[19,:] .= 100.0

learn_periods = []

for i in 1:(length(cal_years)-1)
    yr1 = cal_years[i]
    yr2 = cal_years[i+1]

    # find row indices
    row_idx1 = only(findall(x -> x == yr1, cal_years))
    row_idx2 = only(findall(x -> x == yr2, cal_years))

    # extract vectors (dropping year column)
    row1 = collect(params_df_width_var[row_idx1, :])[2:end]
    row2 = collect(params_df_width_var[row_idx2, :])[2:end]

    # compute differences
    diff = row2 .- row1
    Δ_width_var[i, :] = round.(diff, sigdigits = 6)
    Δ_width_var_pct[i, :] = (diff ./ total_diff) .* 100
    # Collect this learning period as string for datframe labels
    push!(learn_periods, "$(yr1) - $(yr2)")
end
# Add the total learning period (2015 not included since multiple trajectories only began with 2030 caliobration onward)
push!(learn_periods, "2030 - 2300")
width_variance_change = zeros(19,12)

# Put the parameter width changes and percent changes together param by param columnwise
for i in 1:6
    width_variance_change[:,(2*i)-1] = Δ_width_var[:,i]
    width_variance_change[:,(2*i)] = Δ_width_var_pct[:,i]
end


names = ["LearningPeriod","ΔvmThresh","Δ_%_vmThresh","Δ_fricExp","Δ_%_fricExp","Δ_mu_scale","Δ_%_mu_scale","Δ_stiff_scale","Δ_%_stiff_scale","Δ_gamma0","Δ_%_gamma0","Δ_melt_flux","Δ_%_melt_flux"]
width_variance_w_years = hcat(learn_periods, width_variance_change)
df_width_variance_yr_to_yr = DataFrame(width_variance_w_years , names)
# CSV.write("../Data/Parameter_Learning_Tables/yr_to_yr_change_in_width_uncertainty_log_gamma0_$(gamma_0_is_log).csv", df_width_variance_yr_to_yr)

Row,LearningPeriod,ΔvmThresh,Δ_%_vmThresh,Δ_fricExp,Δ_%_fricExp,Δ_mu_scale,Δ_%_mu_scale,Δ_stiff_scale,Δ_%_stiff_scale,Δ_gamma0,Δ_%_gamma0,Δ_melt_flux,Δ_%_melt_flux
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,2030 - 2045,2950.94,69.4426,0.0109701,-935.296,0.044943,98.3006,0.0399945,117.005,0.364371,-1085.09,0.529161,14.3511
2,2045 - 2060,49.035,1.15391,0.000996436,-84.9549,0.0113058,24.7283,0.00455269,13.319,-0.101516,302.313,0.778981,21.1263
3,2060 - 2075,857.073,20.169,0.00453659,-386.784,0.0132468,28.9738,0.00461352,13.4969,-0.188508,561.372,1.47982,40.1333
4,2075 - 2090,346.907,8.16355,0.00717519,-611.748,-0.00394326,-8.62479,0.00473843,13.8624,-0.00993051,29.5728,1.29808,35.2046
5,2090 - 2105,173.415,4.08086,0.0113467,-967.404,0.0291168,63.685,-3.90227e-6,-0.0114162,0.138624,-412.819,0.987398,26.7787
6,2105 - 2120,-222.925,-5.24596,0.00481601,-410.606,0.00548521,11.9974,0.00696103,20.3646,-0.083712,249.292,-0.215089,-5.8333
7,2120 - 2135,-813.782,-19.1502,0.00622987,-531.15,0.00188271,4.11792,-0.00312091,-9.13028,-0.00883196,26.3014,-0.526114,-14.2684
8,2135 - 2150,466.664,10.9817,-0.00203925,173.864,0.00532965,11.6572,0.00929686,27.1981,-0.0113802,33.89,-0.0742345,-2.01327
9,2150 - 2165,1206.95,28.4023,0.00202943,-173.027,-0.00801462,-17.5298,-0.00559898,-16.3799,0.156246,-465.296,-0.665101,-18.0378


In [9]:
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
###############################                SLR Projection Learning Tables             ###########################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################

In [10]:
# All years 2016-2300
all_years_no_gap = collect(range(2016, step=1, length=285))

yrs_dict = Dict{Int64, Int64}()
for (idx,yr) in enumerate(all_years_no_gap)
        yrs_dict[yr] = idx
end


Realizations = [string(i) for i in 1:100];

cal_years = collect(range(2030,step=15,length=19));

In [11]:
# The years of projection considered
chosen_projection_years = [2100,2200,2300]

# Initialize empty arrays (Some entries will be missing due to calibration past year of projection)
all_bounds   = Array{Array{Union{Missing,Tuple{Missing, Missing},Tuple{Float64,Float64},Int64}}}(undef, length(chosen_projection_years))
all_widths   = zeros(3,20)
all_width_variation = zeros(3,19)
# For each projection year considered
for (n, year) in enumerate(chosen_projection_years)
    
    proj_yr_idx = yrs_dict[year]
    proj_bounds = Array{Union{Missing,Float64}}(undef,length(Realizations),19,2)
    proj_widths = zeros(100,19)
    
    # For each of the 100 trajectories considered
    for j in 1:length(Realizations)
        # For each of the 19 calibrating years considered
        for (iter,yr) in enumerate(cal_years[2:end])
                cal_year_matrix = JLD2.load("../Data/Projection_Data/R_$(Realizations[j])/$(Realizations[j])-year$(yr)pred_VAF.jld2",
                    "sample_post_mm_ssp5")
                proj_bounds[j,iter,1] = quantile(cal_year_matrix[proj_yr_idx,:], 0.05)
                proj_bounds[j,iter,2] = quantile(cal_year_matrix[proj_yr_idx,:], 0.95)
                proj_widths[j,iter] = proj_bounds[j,iter,2] - proj_bounds[j,iter,1]
        end
    end

    # Average across the 100 trajectories
    avg_bounds = dropdims(mean(proj_bounds,dims=1),dims=1)
    avg_widths = dropdims(mean(proj_widths,dims=1),dims=1)

    # Compute 66% credible interval of the 90% interval widths across the 100 trajectories
    width_variation_interval = np.quantile(proj_widths, 0.83,axis=0) .- np.quantile(proj_widths,0.17,axis=0) #31
    println(size(width_variation_interval))


    # Load the present day projections
    present_projections = JLD2.load("../Data/Projection_Data/2015_SLR_Projections.jld2","present2015_post_mm_ssp5")
    present_lower = quantile(present_projections[proj_yr_idx,:], 0.05)
    present_upper = quantile(present_projections[proj_yr_idx,:], 0.95)
    present_width = present_upper - present_lower

    # Add present day projections to beginning of lists above
    bounds_w_present = vcat([present_lower;; present_upper], avg_bounds)
    widths_w_present = vcat([present_width], avg_widths);
    # Store the bounds as tuples in one matrix
    slr_tuple_bounds = [(round(bounds_w_present[i,1],sigdigits=6),round(bounds_w_present[i,2],sigdigits=6)) for i in 1:length(bounds_w_present[:,1])]

    # Push this projection years quantities to list containing all the projection years
    all_bounds[n]   = slr_tuple_bounds
    all_widths[n,:]   =  widths_w_present
    all_width_variation[n,:]  =  width_variation_interval
    
end


(19,)
(19,)
(19,)


In [12]:
cal_years = collect(range(2015,step=15,length=20));
cal_years = Int.(cal_years)

# Add the years of calibration as another column
bounds_w_years = hcat(cal_years, reduce(hcat, all_bounds))
widths_w_years = hcat(cal_years, all_widths')
width_var_w_years = hcat(cal_years[2:end],all_width_variation' )

names = ["Latest year of calibration","2100", "2200","2300"]

# Convert to dataframes before saving to disc
slr_df_bounds  = DataFrame(bounds_w_years, names)
slr_df_widths  = DataFrame(widths_w_years, names)
slr_df_width_var = DataFrame(width_var_w_years, names)
allowmissing!(slr_df_bounds) 
allowmissing!(slr_df_widths) 
allowmissing!(slr_df_width_var);

In [13]:
# Remove data that has been calibrated past its projection year (i.e projections for year 2100 with parameters calibrated in 2105)
for yr in names[2:3]
    n_useful = (parse(Int, yr) - 2015) ÷ 15
    slr_df_bounds[n_useful+2:end, yr] .= missing
    slr_df_widths[n_useful+2:end, yr] .= 0
    slr_df_width_var[n_useful+1:end, yr] .= 0    

end

#Calculate total percent decrease of 90% cred interval width
pct_change2100 = ((slr_df_widths[1,2] - slr_df_widths[6,2]) /  slr_df_widths[1,2]) * 100
pct_change2200 = ((slr_df_widths[1,3] - slr_df_widths[13,3]) /  slr_df_widths[1,3]) * 100
pct_change2300 = ((slr_df_widths[1,4] - slr_df_widths[19,4]) /  slr_df_widths[1,4]) * 100

# Make a 1×7 row: first element is label, then pct_change values
new_row = hcat("total_%_decrease", pct_change2100, pct_change2200, pct_change2300)
println(typeof(new_row))
# Convert to DataFrame with the same column names
new_df = DataFrame(new_row, names)


# Append to params_df_widths
append!(slr_df_widths, new_df, promote=true) #22
println(slr_df_widths)

CSV.write("../Data/SLR_Learning_Tables/SLR_90_CI_bounds.csv", slr_df_bounds)
CSV.write("../Data/SLR_Learning_Tables/SLR_90_CI_widths.csv", slr_df_widths)
CSV.write("../Data/SLR_Learning_Tables/SLR_90_CI_variation.csv",  slr_df_width_var)

Matrix{Any}
21×4 DataFrame
 Row │ Latest year of calibration  2100     2200     2300    
     │ Any                         Any      Any      Any     
─────┼───────────────────────────────────────────────────────
   1 │ 2015.0                      5.81829  65.2261  76.6676
   2 │ 2030.0                      4.62033  52.8798  70.4075
   3 │ 2045.0                      4.42315  40.4086  69.8543
   4 │ 2060.0                      4.22244  22.92    67.1312
   5 │ 2075.0                      4.14338  21.8803  66.1392
   6 │ 2090.0                      4.02504  20.5155  61.188
   7 │ 2105.0                      0.0      17.3835  58.2574
   8 │ 2120.0                      0.0      15.5806  56.4196
   9 │ 2135.0                      0.0      14.8113  53.0541
  10 │ 2150.0                      0.0      14.4658  50.1841
  11 │ 2165.0                      0.0      13.6753  42.5954
  12 │ 2180.0                      0.0      12.23    35.005
  13 │ 2195.0                      0.0      10.0191  29.0

"../Data/SLR_Learning_Tables/SLR_90_CI_vaXXXXXriation.csv"

In [14]:
((2300 - 2015) ÷ 15) + 1

20

In [15]:
# CALCULATING THE YEAR TO YEAR CHANGE IN 90% UNCERTAINTY RANGES FOR PROJECTIONS OF THE YEARS 2100, 2200, 2300
proj_years = [2100,2200,2300]
cal_years = collect(range(2015, step=15, length=20))

Δ_proj_widths_yty = Array{Array{Union{Float64, Missing}}}(undef, 6)
for (i,yr) in enumerate(proj_years)
    n_cal_years = ((yr - 2015) ÷ 15) + 1
    # Calculate total learning
    present_row_idx = only(findall(x -> x == 2015, cal_years))
    present_diff = (i == 3) ? slr_df_widths[n_cal_years - 1, i+1] - slr_df_widths[present_row_idx,i+1] : slr_df_widths[n_cal_years, i+1] - slr_df_widths[present_row_idx,i+1]
    abs_change = Array{Union{Float64, Missing}}(undef, 20)
    pct_change = Array{Union{Float64, Missing}}(undef, 20)
    # Calculate the year to year change and percent of total change
    proj_yr_widths = slr_df_widths[!,i+1]
    abs_change[1:19] = proj_yr_widths[2:20] - proj_yr_widths[1:19] #15
    pct_change[1:19] = (abs_change[1:19] ./ present_diff) .* 100
    # Add the total change to the end of the list
    abs_change[20] = present_diff
    # Concatenate to the full matrix
    Δ_proj_widths_yty[2*i-1] = abs_change
    Δ_proj_widths_yty[2*i] = pct_change
        
end


In [16]:
# Labels for datframe rows
learn_periods = vcat(["$(cal_years[i]) - $(cal_years[i+1])" for i in 1:(length(cal_years) - 1)], ["Total learning"])
# Collect labels with data
mat_yty_width_change = hcat(learn_periods, reduce(hcat, Δ_proj_widths_yty))
# We don't care about the change in uncertainty past the point of projection
mat_yty_width_change[6,2] = 0.0
mat_yty_width_change[6,3] = 0.0
mat_yty_width_change[13,4] = 0.0
mat_yty_width_change[13,5] = 0.0
mat_yty_width_change[19,6] = 0.0
mat_yty_width_change[19,7] = 0.0
# Column labels
names = ["Learning_Period","Δ_2100","Δ_%_2100","Δ_2200","Δ_%_2200", "Δ_2300","Δ_%_2300"]
df_yty_width_change  = DataFrame(mat_yty_width_change, names)
CSV.write("../Data/SLR_Learning_Tables/yr_to_yr_90_CI_width_change.csv",   df_yty_width_change)

"../Data/SLR_Learning_Tables/yr_to_yr_90_CI_widthXCXXXX_change.csv"

In [17]:
df_yty_width_change

Row,Learning_Period,Δ_2100,Δ_%_2100,Δ_2200,Δ_%_2200,Δ_2300,Δ_%_2300
,Any,Any,Any,Any,Any,Any,Any
1,2015 - 2030,-1.19796,66.8038,-12.3463,22.3636,-6.26004,9.82103
2,2030 - 2045,-0.197182,10.9958,-12.4713,22.59,-0.553257,0.867974
3,2045 - 2060,-0.200712,11.1927,-17.4885,31.6781,-2.7231,4.27212
4,2060 - 2075,-0.0790525,4.40833,-1.0397,1.88327,-0.991968,1.55624
5,2075 - 2090,-0.118345,6.59948,-1.3648,2.47214,-4.95117,7.76762
6,2090 - 2105,0.0,0.0,-3.132,5.67319,-2.93064,4.59772
7,2105 - 2120,0.0,-0.0,-1.80292,3.26575,-1.83781,2.88323
8,2120 - 2135,0.0,-0.0,-0.769318,1.39352,-3.3655,5.27994
9,2135 - 2150,0.0,-0.0,-0.345495,0.625817,-2.86996,4.50251


In [18]:
println(sum(df_yty_width_change[1:4, 5]))

78.51500443127654


In [19]:
# CALCULATING THE YEAR TO YEAR CHANGE IN 90% UNCERTAINTY RANGE STANDARD DEVIATION FOR PROJECTIONS OF THE YEARS 2100, 2200, 2300
proj_years = [2100,2200,2300]
cal_years = collect(range(2030, step=15, length=19))

Δ_proj_width_var_yty = Array{Array{Union{Float64, Missing}}}(undef, 6)
for (i,yr) in enumerate(proj_years)
    n_cal_years = ((yr - 2030) ÷ 15) + 1
    # Calculate total learning
    present_row_idx = only(findall(x -> x == 2030, cal_years))
    present_diff = (i == 3) ? slr_df_width_var[n_cal_years - 1, i+1] - slr_df_width_var[present_row_idx,i+1] : slr_df_width_var[n_cal_years, i+1] - slr_df_width_var[present_row_idx,i+1]
    abs_change = Array{Union{Float64, Missing}}(undef, 19)
    pct_change = Array{Union{Float64, Missing}}(undef, 19)
    # Calculate the year to year change and percent of total change
    proj_yr_width_var = slr_df_width_var[!,i+1]
    abs_change[1:18] = proj_yr_width_var[2:19] - proj_yr_width_var[1:18]
    pct_change[1:18] = (abs_change[1:18] ./ present_diff) .* 100
    # Add the total change to the end of the list
    abs_change[19] = present_diff
    # Concatenate to the full matrix
    Δ_proj_width_var_yty[2*i-1] = abs_change
    Δ_proj_width_var_yty[2*i] = pct_change
        
end


In [20]:
# Labels for datframe rows
learn_periods = vcat(["$(cal_years[i]) - $(cal_years[i+1])" for i in 1:(length(cal_years) - 1)], ["Total learning"])
# Collect labels with data
mat_yty_std_change = hcat(learn_periods, reduce(hcat, Δ_proj_width_var_yty))
# We don't care about the change in uncertainty variation past the point of projection
mat_yty_std_change[5,2] = 0.0
mat_yty_std_change[5,3] = 0.0
mat_yty_std_change[12,4] = 0.0
mat_yty_std_change[12,5] = 0.0
mat_yty_std_change[18,6] = 0.0
mat_yty_std_change[18,7] = 0.0
# Column labels
names = ["Learning_Period","Δ_2100","Δ_%_2100","Δ_2200","Δ_%_2200", "Δ_2300","Δ_%_2300"]
df_yty_std_change  = DataFrame(mat_yty_std_change, names)
CSV.write("../Data/SLR_Learning_Tables/yr_to_yr_90_CI_width_variation_change.csv",   df_yty_std_change)

"../Data/SLR_Learning_Tables/yr_to_yr_90_CI_widthXXXX_variation_change.csv"